In [1]:
import pandas as pd

In [5]:
hands_used  = pd.read_csv("signs_f.csv")

In [15]:
two_hand = hands_used[hands_used.Both_Hands.notnull()].Sign.values
one_hand = hands_used[hands_used.Both_Hands.isnull()].Sign.values

In [16]:
import glob, os
import re

In [17]:
glob_list = []

#loop through subject folders and glob
for subject in range(25):
    glob_list.append(glob.glob(str(subject) + "\\[A-Z0-9]*.csv"))
    
#function to flatten glob
flatten = lambda l: [item for sublist in l for item in sublist]
glob_list = flatten(glob_list)

In [19]:
#make a list of tuples of hand pairings
lr_pairs = list(zip(*[iter(glob_list)]*2))

# remove all blue files
for i in range(25):
    lr_pairs.remove((str(i)+'\\Blue_Left.csv', str(i)+'\\Blue_Right.csv'))
# remove "bad" from subject 19
lr_pairs.remove(('19\\Bad_Left.csv', '19\\Bad_Right.csv'))

In [25]:
df_list = []

for pair in lr_pairs:
    df_left = pd.read_csv(pair[0], index_col=None).drop(['Unnamed: 0'], axis = 1)
    df_right = pd.read_csv(pair[1], index_col=None).drop(['Unnamed: 0'], axis = 1)
    
    #rename columns
    df_left = df_left.add_prefix('left')
    df_right = df_right.add_prefix('right')
    
    #merge
    df = pd.merge(df_left, df_right, left_on='leftTime', right_on='rightTime').drop('rightTime', axis = 1)

    #covert fist column to time object
    df['leftTime'] = pd.to_datetime(df['leftTime'].str[:-3], format = '%H:%M:%S.%f')
    
    #difference between rows
    df = df.diff().iloc[1:]
    df['leftTime'] = df['leftTime'].dt.total_seconds()
    
    df.rename(columns={'leftTime':'time'}, inplace=True)
    
    #add sign and subject using regex of file name
    subject_sign = re.split(r'\\', re.findall('^[^_]+(?=_)', pair[0])[0])
    df.insert(loc = 0, column = 'Subject', value = subject_sign[0])
    df.insert(loc = 0, column = 'Sign', value = subject_sign[1])
    
#this makes velocity columns

#     for col in df.iloc[:,2:]:
#         df[col + ' velocity'] = df[col]/df['Time']
    
    df_list.append(df)

In [218]:
from sklearn.base import BaseEstimator, TransformerMixin

class hand_selection(BaseEstimator, TransformerMixin):
    
    def __init__(self, hand_list, drop_left=False):
        self.hand_list = hand_list
        self.drop_left = drop_left
        
    def transform(self, df_list):
        if not self.drop_left:
            subset = [df for df in df_list if df.Sign.values[0] in self.hand_list]
        else:
            subset = [df.drop(df.filter(regex='left').columns, axis=1) \
                      for df in df_list \
                      if df.Sign.values[0] in self.hand_list]
            
        return subset
    
    def fit(self, df_list, y = None):
        return self

In [253]:
from sklearn.pipeline import Pipeline, FeatureUnion

one_hand_pipeline = Pipeline([
    ('one hand', hand_selection(one_hand, drop_left = True))
])

two_hand_pipeline = Pipeline([
    ('two hand', hand_selection(two_hand, drop_left = False))
])

In [254]:
blah = one_hand_pipeline.transform(df_list)

In [255]:
blah[0]

,Sign,Subject,time,right thumbProximal_L_X,right thumbProximal_L_Y,right thumbProximal_L_Z,right thumbDistal_L_X,right thumbDistal_L_Y,right thumbDistal_L_Z,right thumbEF_L_X,...,right armFrontRadius_L_Z,right armFrontUlna_L_X,right armFrontUlna_L_Y,right armFrontUlna_L_Z,right armBackLateral_L_X,right armBackLateral_L_Y,right armBackLateral_L_Z,right armBackMedial_L_X,right armBackMedial_L_Y,right armBackMedial_L_Z
1,0,0,0.028,-0.000100,0.000042,-0.000299,-0.000078,0.000039,-3.071000e-04,-0.000069,...,-0.000321,-0.000166,0.000114,-0.000298,-8.360000e-05,-0.000393,-0.000142,-0.000071,-0.000331,-0.000119
2,0,0,0.036,-0.000002,0.000199,-0.000187,0.000015,0.000225,-1.838000e-04,0.000015,...,-0.000235,-0.000095,0.000121,-0.000193,4.000000e-05,-0.000276,-0.000075,0.000035,-0.000261,-0.000034
3,0,0,0.033,-0.000035,0.000296,-0.000250,-0.000073,0.000296,-2.370000e-04,-0.000106,...,-0.000251,-0.000084,0.000163,-0.000204,-1.250000e-04,-0.000481,0.000008,-0.000147,-0.000522,0.000054
4,0,0,0.034,-0.000138,0.000626,-0.000216,-0.000229,0.000609,-1.911000e-04,-0.000309,...,-0.000213,-0.000179,0.000077,-0.000056,1.000000e-07,-0.000630,0.000164,-0.000104,-0.000869,0.000321
5,0,0,0.036,-0.000138,0.000650,-0.000026,-0.000236,0.000633,7.000000e-07,-0.000279,...,-0.000001,0.000016,0.000356,0.000150,5.750000e-04,0.000152,0.000179,0.000504,0.000025,0.000330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,0,0,0.042,0.001564,0.000210,0.000265,0.001507,-0.000664,5.004000e-04,0.001807,...,0.000295,0.001436,0.000441,0.000373,-8.165000e-04,-0.000436,-0.000397,-0.000785,-0.000404,-0.000319
686,0,0,0.046,0.001785,-0.000203,0.000642,0.001734,-0.001654,9.838000e-04,0.002163,...,0.000138,0.002025,0.000737,0.000454,-1.395300e-03,-0.001677,-0.000466,-0.001304,-0.001543,-0.000149
687,0,0,0.043,0.002699,0.000931,0.000509,0.002810,0.000097,8.319000e-04,0.003130,...,-0.000251,0.002495,0.001583,-0.000102,-1.360000e-03,-0.000036,-0.001245,-0.001321,0.000028,-0.001097
688,0,0,0.059,0.005148,0.001957,0.000299,0.005855,0.001178,1.313300e-03,0.006791,...,-0.000735,0.005017,0.002821,0.000149,-2.756600e-03,-0.002480,-0.001475,-0.001811,-0.002021,-0.000591
